# Modelling SEI growth on particle cracks

This notebook provides a short demonsration of how the SEI and particle mechanics submodels can be combined to simulate SEi growth on particle cracks.

In [1]:
%pip install pybamm -q    # install PyBaMM if it is not installed
import pybamm
import matplotlib.pyplot as plt
import numpy as np

You should consider upgrading via the '/home/sokane/PyBaMM/env/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


Define two models. In model1, the only degradation mechanism is solvent-diffusion limited SEI growth. model2 includes the same SEI growth mechanism but also includes particle cracking and SEI growth on the cracks. The SEI model is run twice: once on the initial surface and once on the cracks. The equations for SEI on cracks are reported by O'Kane et al. [8]

In [2]:
model1 = pybamm.lithium_ion.DFN({"SEI": "none"})
model2 = pybamm.lithium_ion.DFN({
    "particle mechanics": "swelling and cracking",
    "SEI": "solvent-diffusion limited",
    "SEI on cracks": "true",
})

Depending on the parameter set being used, the particle cracking model can require a large number of mesh points inside the particles to be numerically stable.

In [3]:
parameter_values = pybamm.ParameterValues("OKane2022")
var_pts = {
    "x_n": 20,  # negative electrode
    "x_s": 20,  # separator 
    "x_p": 20,  # positive electrode
    "r_n": 120,  # negative particle
    "r_p": 100,  # positive particle
}

Solve the models with and without cracking for a 1C discharge.

In [4]:
experiment = pybamm.Experiment(["Discharge at 1C until 2.5 V"])
sim1 = pybamm.Simulation(model1, parameter_values=parameter_values, experiment=experiment, var_pts=var_pts)
sol1 = sim1.solve()
sim2 = pybamm.Simulation(model2, parameter_values=parameter_values, experiment=experiment, var_pts=var_pts)
sol2 = sim2.solve()

In [ ]:
t1 = sol1["Time [s]"].entries
V1 = sol1["Terminal voltage [V]"].entries
SEI1 = sol1["Loss of capacity to SEI [A.h]"].entries
t2 = sol2["Time [s]"].entries
V2 = sol2["Terminal voltage [V]"].entries
SEI2 = sol2["Loss of capacity to SEI [A.h]"].entries + sol2["Loss of capacity to SEI on cracks [A.h]"].entries

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(18,4))
ax1.plot(t1,V1,label="without cracking")
ax1.plot(t2,V2,label="with cracking")
ax1.set_xlabel("Time [s]")
ax1.set_ylabel("Terminal voltage [V]")
ax1.legend()
ax2.plot(t1,SEI1,label="without cracking")
ax2.plot(t2,SEI2,label="with cracking")
ax2.set_xlabel("Time [s]")
ax2.set_ylabel("Loss of capacity to SEI [A.h]")
ax2.legend()
plt.show()

The SEI on cracks consumes far more capacity than the SEI on the initial surface, in agreement with the literature.

In [ ]:
pybamm.print_citations()